# Dataset

In [ ]:
!nvidia-smi

Wed Apr 24 14:23:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   46C    P0              27W /  72W |  22525MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install wandb
!wandb login
import wandb

%env WANDB_PROJECT=webnlg-fintune-gemma-2b-it

wandb: Currently logged in as: rpucs. Use `wandb login --relogin` to force relogin
env: WANDB_PROJECT=webnlg-fintune-gemma-2b-it


In [ ]:
!pip3 install -q -U datasets
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers
!pip3 install -q -U Cython
import os
os._exit(0)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 29.0 MB/s eta 0:00:00


In [ ]:
import random
import tqdm
import datasets
from datasets import load_dataset

In [ ]:
import os
import torch
import bitsandbytes as bnb
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, setup_chat_format
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model, PeftConfig

from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [ ]:
dataset = load_dataset("web_nlg", "release_v3.0_en")
train, test, dev = dataset["train"], dataset["test"], dataset["dev"]

Generating train split:   0%|          | 0/13211 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1667 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5713 [00:00<?, ? examples/s]

In [ ]:
train[0]

{'category': 'Airport',
 'size': 1,
 'eid': 'Id1',
 'original_triple_sets': {'otriple_set': [['Aarhus_Airport | cityServed | "Aarhus, Denmark"@en']]},
 'modified_triple_sets': {'mtriple_set': [['Aarhus_Airport | cityServed | "Aarhus, Denmark"']]},
 'shape': '(X (X))',
 'shape_type': 'NA',
 'lex': {'comment': ['good', 'good'],
  'lid': ['Id1', 'Id2'],
  'text': ['The Aarhus is the airport of Aarhus, Denmark.',
   'Aarhus Airport serves the city of Aarhus, Denmark.'],
  'lang': ['', '']},
 'test_category': '',
 'dbpedia_links': [],
 'links': []}

In [ ]:
def preprocess_webnlg(dataset):
  """
  Create a new dataset with the columns of 'entryID', 'text', 'relations'.
  From one webnlg datapoint multiple rows can be generated as there may be more than one variation of the sentences inside the 'text' field.
  Relation points are sorted in abc order.
  """
  i=0
  new_dataset_rows = {
      'entryID':[],
      'text':[],
      'relations':[]
  }
  for datapoint in dataset:
    relations = datapoint['modified_triple_sets']['mtriple_set'][0]
    relations = [ [point.strip() for point in relation.split('|')] for relation in relations]
    relations = sort_them(relations)

    texts = datapoint['lex']['text']
    for text in texts:
      new_dataset_rows['text'].append(text)
      new_dataset_rows['relations'].append(relations)
      new_dataset_rows['entryID'].append(i)

    i += 1

  return datasets.Dataset.from_dict(new_dataset_rows)

def create_harder_dataset(dataset, select_percentage=1.):
  """
  Create a harder dataset with more sentences that are more diverse.
  """
  # shuffle dataset
  dataset = dataset.shuffle()
  # select
  size = int(len(dataset)*select_percentage)
  dataset = dataset.select(range(size))
  new_dataset_rows = {
      'entryID':[],
      'text':[],
      'relations':[]
  }
  # sample indecies
  list_of_indecies = range(len(dataset))
  for i in tqdm.tqdm(range(len(dataset))):
    num_of_indecies = random.choices([1,2,3,4,5], weights=[0.4, 0.2, 0.2, 0.1, 0.1])
    num_of_indecies = num_of_indecies[0]
    current_indecies = random.sample(list_of_indecies, k=num_of_indecies)
    # create new dataset
    selected_data = dataset.select(current_indecies)
    _text = " ".join(selected_data["text"])
    _relations = [item for l_item in selected_data["relations"] for item in l_item]
    _relations = sort_them(_relations)
    new_dataset_rows["entryID"].append(i)
    new_dataset_rows["text"].append(_text)
    new_dataset_rows["relations"].append(_relations)

  return datasets.Dataset.from_dict(new_dataset_rows)


def sort_them(relations):
  relations = sorted(relations, key=lambda x : x[2])
  relations = sorted(relations, key=lambda x : x[1])
  relations = sorted(relations, key=lambda x : x[0])
  return relations

def formatting_prompts_func(dataset):
    output_texts = []
    for datapoint in dataset:
        text = f"### Text: {datapoint['text']}\n ### Relations: {' '.join([f'<entry><head>{rel[0]}<rel>{rel[1]}<tail>{rel[2]}' for rel in datapoint['relations']])}<eos>"
        output_texts.append(text)
    return output_texts

In [ ]:
train, test, dev = preprocess_webnlg(train), preprocess_webnlg(test), preprocess_webnlg(dev)
train, test, dev = create_harder_dataset(train), create_harder_dataset(test), create_harder_dataset(dev)

rows = formatting_prompts_func(train)
train = train.add_column("prompt", rows)

rows = formatting_prompts_func(test)
test = test.add_column("prompt", rows)

rows = formatting_prompts_func(dev)
dev = dev.add_column("prompt", rows)
print(f"""
Length of train: {len(train)}
Length of test: {len(test)}
Length of dev: {len(dev)}
""")

100%|██████████| 4464/4464 [00:07<00:00, 595.34it/s]



Length of train: 35426
Length of test: 7305
Length of dev: 4464



In [ ]:
point = train[random.sample(range(len(train)), k=1)]
print(point["entryID"])
print("\n".join(point["text"][0].split(". ")))
print(point["relations"])

[5619]
The US town of Arlington is found in the state of Texas, where the capital is Austin
Anaheim covers an area of 131.6 square kilometres and is located in Orange County, California, United States
The United States House of Representatives leads Anaheim
Akron is in Summity County, Ohio, uS
The leader of the US is the President and Washington DC is the capital
Native Americans are a US Ethnic group
Julia Morgan (born in San Francisco) was the architect of the Asilomar Conference grounds, The Los Angeles Herald Examiner building, The Riverside Art Museum and Hearst Castle
ENAIRE is the operating organisation for Adolfo Suarez Madrid-Barajas airport in Madrid.
[[['Adolfo_Suárez_Madrid–Barajas_Airport', 'operatingOrganisation', 'ENAIRE'], ['Akron,_Ohio', 'country', 'United_States'], ['Akron,_Ohio', 'isPartOf', 'Summit_County,_Ohio'], ['Anaheim,_California', 'areaTotal', '131.6 (square kilometres)'], ['Anaheim,_California', 'country', 'United_States'], ['Anaheim,_California', 'isPartOf'

# Training

## Prompt
| # Text: [text]<br>
| # Relations: |entry||head|[head]|rel|[rel]|tail|[tail]\\n


In [ ]:
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
      lora_module_names.remove('lm_head')
  return list(lora_module_names)


In [ ]:
model_id = "google/gemma-2b-it"
new_model = "gemma-2b-relextract-webnlg-final" #Name of the model you will be pushing to huggingface model hub

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quent_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          add_eos_token=True
                                          )
tokenizer.padding_side = "right"

#print("Tokeinzer default eos: ", tokenizer.eos_token)
#tokenizer.eos_token = '<eos>'
#tokenizer.pad_token = '<pad>'
#tokenizer.bos_token = '<bos>'
#tokenizer.add_special_tokens({"additional_special_tokens": ['<entry>', '<head>', '<rel>', '<tail>',
#                                                            '### Text:', '### Relations:'
#                                                            ]})
#model.resize_token_embeddings(len(tokenizer))

#tokenizer.pad_token = tokenizer.eos_token

Unused kwargs: ['bnb_4bit_quent_type']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# https://github.com/huggingface/trl/issues/588
# https://huggingface.co/docs/trl/v0.7.2/en/sft_trainer#using-tokenids-directly-for-responsetemplate
#response_template = "\n ### Relations:"
#first = tokenizer.encode(response_template, add_special_tokens=False)[2:]
#second = tokenizer.encode("Hi!\n ### Relations:", add_special_tokens=False)
#print(first, second)

In [ ]:
max_steps=5000
logging_steps=1
run_name="run-5000-steps-nospecialtoken-it"


model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

response_template_string = "\n ### Relations:"
response_template = tokenizer.encode(response_template_string, add_special_tokens=False)[2:]
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

modules = find_all_linear_names(model)
lora_config = LoraConfig(
  r=64,
  lora_alpha=32,
  target_modules=modules,
  lora_dropout=0.05,
  bias="none",
  task_type="CASUAL_LM"
)

model = get_peft_model(model, lora_config)

trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")


trainer = SFTTrainer(
      model=model,
      train_dataset=train,
      eval_dataset=test,
      dataset_text_field="prompt",
      peft_config=lora_config,
      args=transformers.TrainingArguments(
          optim="paged_adamw_8bit",
          per_device_train_batch_size=2,
          gradient_accumulation_steps=4,
          warmup_steps=0.03,
          learning_rate=2e-4,
          max_steps=max_steps,
          logging_steps=logging_steps,
          report_to="wandb",
          run_name=run_name,
          output_dir="outputs",
          #save_steps=10
          #num_train_epochs=2,
          #save_strategy="epoch",
      ),
      #data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
      data_collator=collator,
      #max_seq_length=10_000,
      packing=False
  )

trainer.train()
wandb.finish()

# Upload
print("Uploading {} to hub".format(new_model))
trainer.model.save_pretrained(new_model)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

Trainable: 78446592 | total: 2584619008 | Percentage: 3.0351%


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/35426 [00:00<?, ? examples/s]

Map:   0%|          | 0/7305 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
wandb: Currently logged in as: rpucs. Use `wandb login --relogin` to force relogin


...

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rPucs/gemma-2b-relextract-webnlg-final/commit/c060334d109cc55879cec177e77e3ed2910fa4ef', commit_message='Upload tokenizer', commit_description='', oid='c060334d109cc55879cec177e77e3ed2910fa4ef', pr_url=None, pr_revision=None, pr_num=None)

Uploading gemma-2b-relation-extracter-webnlg to hub


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rPucs/gemma-2b-relation-extracter-webnlg/commit/b5b790da7b24c3dd7090388800c601474b409752', commit_message='Upload tokenizer', commit_description='', oid='b5b790da7b24c3dd7090388800c601474b409752', pr_url=None, pr_revision=None, pr_num=None)

# Test out model

In [ ]:
#config = PeftConfig.from_pretrained("rPucs/gemma-2b-relextract-nospecialtoken-webnlg")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quent_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained("rPucs/gemma-2b-relextract-webnlg-final", quantization_config=bnb_config, device_map={"":0})

tokenizer = AutoTokenizer.from_pretrained("rPucs/gemma-2b-relextract-webnlg-final",
                                          add_eos_token=True
                                          )

Unused kwargs: ['bnb_4bit_quent_type']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

In [ ]:


def inference(text, model, tokenizer):
  template = "### Text: {}\n ### Relations: ".format(text)
  encoded = encodeds = tokenizer(template, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to("cuda:0")
  generated_ids = model.generate(**model_inputs, max_new_tokens=300, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
  return decoded.split("### Relations: ")[1]

idx = random.sample(range(len(test)), k=1)[0]
print("True: \n{}\n\n".format(test["prompt"][idx]))
text = test["text"][idx]

relations = inference(text, model, tokenizer)
print("")
print("Relations: {}".format(relations))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


True: 
### Text: The main ingredients of Bionico are chopped fruits, sour cream, condensed milk, granola, shredded coconut and raisins. Bionico (a dish served for dessert) is a food found in Mexico. The currency in Mexico is the Mexican peso inhabited by the Mexicans led by Silvano Aureoles Conejo.
 ### Relations: <entry><head>Bionico<rel>country<tail>Mexico <entry><head>Bionico<rel>course<tail>Dessert <entry><head>Bionico<rel>mainIngredient<tail>"Chopped Fruits, Sour Cream, Condensed Milk, Granola, Shredded Coconut, Raisins" <entry><head>Mexico<rel>currency<tail>Mexican_peso <entry><head>Mexico<rel>demonym<tail>Mexicans <entry><head>Mexico<rel>leader<tail>Silvano_Aureoles_Conejo<eos>



Relations: <eos><entry><head>Bionico<rel>country<tail>Mexico <entry><head>Bionico<rel>course<tail>Dessert <entry><head>Bionico<rel>ingredient<tail>Condensed_milk <entry><head>Bionico<rel>mainIngredient<tail>"Chopped Fruits, Sour Cream, Condensed Milk, Granola, Shredded Coconut, Raisins" <entry><head>Me

In [ ]:
text = "Prucs Ákos is a citizen of Dunaszekcső. He studied Computer Science and likes to play videogames."

relations = inference(text, model, tokenizer)
print("")
print("Relations: {}".format(relations))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



Relations: <eos><entry><head>Prucs_Ákos<rel>almaMater<tail>Computer_science <entry><head>Prucs_Ákos<rel>birthPlace<tail>Dunaszekső <entry><head>Prucs_Ákos<rel>interest<tail>Video_games<eos>
